**Dominika Kaczmarska**
 numer indeksu: 116977

# Analiza czynników personalnych wpływających na depresję, niepokój i stres

### 1. Opis problemu
Depresja, niepokój i stres to stany, które dotykają wiele osób. W celu określenia skali tych stanów są stworzone różne testy, a jednym z nich jest Depression Anxiety Stress Scales (DASS). W wersji online tego testu oprócz pytań określających stopień zaburzeń, pojawiły się pytania personalne takie jak wiek, płeć, orientacja. 

Analizowany zbiór zawiera dane osób, które wypełniały test w latach 2017-2019 oraz wyraziły zgodę na wykorzystanie wyników do badań.

Analiza tego zbioru pozwoli określić charakterystykę tych zaburzeń.


_Źródło danych:_ https://www.kaggle.com/yamqwe/depression-anxiety-stress-scales

### 2. Przygotowanie danych

###### Wczytanie danych z pliku CSV

In [0]:
d = spark.read\
  .format("csv")\
  .options(
    inferSchema="true",
    header="true",
    delimiter="	")\
  .load("/FileStore/tables/data.csv")

Badanie zawiera 42 pytania dotyczące zaburzeń, na które były do wyboru cztery odpowiedzi numeryczne:
* 1 - zupełnie mnie nie dotyczy
* 2 - dotyczy mnie w pewnym stopniu, albo dotyczy tylko czasami
* 3 - dotyczy mnie w znaczącym stopniu, albo dotyczy mnie często
* 4 - zdecydowanie mnie dotyczy, albo dotyczy mnie prawie zawsze

Każde pytanie przypisane jest do jednego z trzech zaburzeń - depresji, stresu lub niepokojum - a zsumowanie odpowiedzi dla danego zaburzenia pozwala określić jego stopień.

###### Zsumowanie odpowiedzi na pytania w celu określenia stopnia zaburzenia

In [0]:
d2 = d.withColumn("Depresja", d.Q3A + d.Q5A + d.Q10A + d.Q13A + d.Q16A + d.Q17A + d.Q21A + d.Q24A + d.Q26A + d.Q31A + d.Q34A + d.Q37A + d.Q38A + d.Q42A)\
    .withColumn("Niepokój", d.Q2A + d.Q4A + d.Q7A + d.Q9A + d.Q15A + d.Q19A + d.Q20A + d.Q23A + d.Q25A + d.Q28A + d.Q30A + d.Q36A + d.Q40A + d.Q41A)\
    .withColumn("Stres", d.Q1A + d.Q6A + d.Q8A + d.Q11A + d.Q12A + d.Q14A + d.Q18A + d.Q22A + d.Q27A + d.Q29A + d.Q32A + d.Q33A + d.Q35A + d.Q39A)

###### Wybór interesujących nas kolumn

Kolumny, które będą przez nas używane w analizie to:
* Depresja 
* Niepokój 
* Stres 
* Płeć 
* Wiek
* Wyznanie
* Orientacja
* Stan_cywilny
* Wykształcenie

In [0]:
d3 = d2.select("Depresja", "Niepokój", "Stres", "gender", "age", "religion", "orientation", "married", "education")\
        .toDF("Depresja", "Niepokój", "Stres", "Płeć", "Wiek", "Wyznanie", "Orientacja", "Stan_cywilny", "Wykształcenie")

In [0]:
d3.display()

Depresja,Niepokój,Stres,Płeć,Wiek,Wyznanie,Orientacja,Stan_cywilny,Wykształcenie
41,48,54,2,16,12,1,1,2
38,31,41,2,16,7,0,1,2
53,26,31,2,17,4,3,1,2
30,31,30,2,13,4,5,1,1
46,54,43,2,19,10,1,1,3
27,20,26,2,20,4,1,1,2
45,33,28,2,17,7,2,1,2
18,18,20,2,29,2,2,1,4
49,53,47,1,16,12,2,1,2
56,42,48,2,18,2,2,1,1


W celu łatwiejszej analizy danych zamienimy zmienne liczbowe przedstawiające stopień zaburzeń na zmienne kategoryczne (przedziały dla danych kategorii są zdefiniowane w teście z którego pochodzą dane) oraz zmienne Płeć, Religia, Orientacja, Rada, Stan_cywilny, Wykształcenie na zmienne tekstowe (zgodnia z kluczem podanym w dokumentacji źródła danych).

In [0]:
def Depresja(x):
    if x <= 24:
        return "Brak"
    elif x >= 25 and x <= 27:
        return "Lekki"
    elif x >= 28 and x <= 34:
        return "Średni"
    elif x >= 35 and x <= 41:
        return "Poważny"
    else:
        return "Bardzo poważny"

In [0]:
def Niepokój(x):
    if x <= 21:
        return "Brak"
    elif x >= 22 and x <= 23:
        return "Lekki"
    elif x >= 24 and x <= 28:
        return "Średni"
    elif x >= 29 and x <= 33:
        return "Poważny"
    else:
        return "Bardzo poważny"

In [0]:
def Stres(x):
    if x <= 28:
        return "Brak"
    elif x >= 29 and x <= 32:
        return "Lekki"
    elif x >= 33 and x <= 39:
        return "Średni"
    elif x >= 41 and x <= 47:
        return "Poważny"
    else:
        return "Bardzo poważny"

In [0]:
from pyspark.sql.types import *

In [0]:
udf_Depresja = udf(Depresja, StringType())
udf_Niepokój = udf(Niepokój, StringType())
udf_Stres = udf(Stres, StringType())

In [0]:
d4 = d3.withColumn("Depresja", udf_Depresja(d3.Depresja))
d4 = d4.withColumn("Niepokój", udf_Niepokój(d3.Niepokój))
d4 = d4.withColumn("Stres", udf_Stres(d3.Stres))

In [0]:
import pyspark.sql.functions as f

In [0]:
d4 = d4.withColumn("Płeć",
                  f.when(f.col("Płeć") == 1, "Mężczyzna")\
                   .when(f.col("Płeć") == 2, "Kobieta")\
                   .when(f.col("Płeć") == 3, "Inna")
                  )

In [0]:
d4 = d4.withColumn("Wyznanie",
                  f.when(f.col("Wyznanie") == 1, "Agnostyk")\
                   .when(f.col("Wyznanie") == 2, "Ateista")\
                   .when(f.col("Wyznanie") == 3, "Buddysta")\
                   .when(f.col("Wyznanie") == 4, "Chrześcijanin (Katolik)")\
                   .when(f.col("Wyznanie") == 5, "Chrześcijanin (Mormon)")\
                   .when(f.col("Wyznanie") == 6, "Chrześcijanin (Protestant)")\
                   .when(f.col("Wyznanie") == 7, "Chrześcijanin (Inny odłam)")\
                   .when(f.col("Wyznanie") == 8, "Hindus")\
                   .when(f.col("Wyznanie") == 9, "Żyd")\
                   .when(f.col("Wyznanie") == 10, "Muzułmanin")\
                   .when(f.col("Wyznanie") == 11, "Sikh")\
                   .when(f.col("Wyznanie") == 12, "Inne")
                  )

In [0]:
d4 = d4.withColumn("Orientacja",
                  f.when(f.col("Orientacja") == 1, "Heteroseksualizm")\
                   .when(f.col("Orientacja") == 2, "Biseksualizm")\
                   .when(f.col("Orientacja") == 3, "Homoseksualizm")\
                   .when(f.col("Orientacja") == 4, "Aseksualizm")\
                   .when(f.col("Orientacja") == 5, "Inna")
                  )

In [0]:
d4 = d4.withColumn("Stan_cywilny",
                  f.when(f.col("Stan_cywilny") == 1, "Panna/Kawaler")\
                   .when(f.col("Stan_cywilny") == 2, "Zamężna/Żonaty")\
                   .when(f.col("Stan_cywilny") == 3, "Rozwiedziona/Rozwiedziony")
                  )

In [0]:
d4 = d4.withColumn("Wykształcenie",
                f.when(f.col("Wykształcenie") == 1, "Podstawowe")\
                 .when(f.col("Wykształcenie") == 2, "Średnie")\
                 .when(f.col("Wykształcenie") == 3, "Wyższe 1 stopnia")\
                 .when(f.col("Wykształcenie") == 4, "Wyższe 2 stopnia")
                )

In [0]:
d4.display()

Depresja,Niepokój,Stres,Płeć,Wiek,Wyznanie,Orientacja,Stan_cywilny,Wykształcenie
Poważny,Bardzo poważny,Bardzo poważny,Kobieta,16,Inne,Heteroseksualizm,Panna/Kawaler,Średnie
Poważny,Poważny,Poważny,Kobieta,16,Chrześcijanin (Inny odłam),null,Panna/Kawaler,Średnie
Bardzo poważny,Średni,Lekki,Kobieta,17,Chrześcijanin (Katolik),Homoseksualizm,Panna/Kawaler,Średnie
Średni,Poważny,Lekki,Kobieta,13,Chrześcijanin (Katolik),Inna,Panna/Kawaler,Podstawowe
Bardzo poważny,Bardzo poważny,Poważny,Kobieta,19,Muzułmanin,Heteroseksualizm,Panna/Kawaler,Wyższe 1 stopnia
Lekki,Brak,Brak,Kobieta,20,Chrześcijanin (Katolik),Heteroseksualizm,Panna/Kawaler,Średnie
Bardzo poważny,Poważny,Brak,Kobieta,17,Chrześcijanin (Inny odłam),Biseksualizm,Panna/Kawaler,Średnie
Brak,Brak,Brak,Kobieta,29,Ateista,Biseksualizm,Panna/Kawaler,Wyższe 2 stopnia
Bardzo poważny,Bardzo poważny,Poważny,Mężczyzna,16,Inne,Biseksualizm,Panna/Kawaler,Średnie
Bardzo poważny,Bardzo poważny,Bardzo poważny,Kobieta,18,Ateista,Biseksualizm,Panna/Kawaler,Podstawowe


### 3. Analiza eksploracyjna danych

In [0]:
order = spark.createDataFrame([[1, "Brak"],
                               [2, "Lekki"],
                               [3, "Średni"],
                               [4, "Poważny"],
                               [5, "Bardzo poważny"]],
                              schema=["kolejność", "Poziom"])

In [0]:
order_edu = spark.createDataFrame([[1, "Podstawowe"],
                               [2, "Średnie"],
                               [3, "Wyższe 1 stopnia"],
                               [4, "Wyższe 2 stopnia"]],
                              schema=["kolejność_wykształcenie", "wykszt"])

In [0]:
d5 = d4.groupBy("Depresja").count().withColumn("Zaburzenie", f.lit("Depresja"))\
        .union(d4.groupBy("Niepokój").count().withColumn("Zaburzenie", f.lit("Niepokój")))\
        .union(d4.groupBy("Stres").count().withColumn("Zaburzenie", f.lit("Stres")))\
    .join(order, on = (f.col("Poziom")==f.col("Depresja")), how = "left")\
    .orderBy(f.col("kolejność"))

In [0]:
d5.display()

Depresja,count,Zaburzenie,kolejność,Poziom
Brak,9728,Niepokój,1,Brak
Brak,9771,Depresja,1,Brak
Brak,11800,Stres,1,Brak
Lekki,2764,Niepokój,2,Lekki
Lekki,2871,Depresja,2,Lekki
Lekki,4921,Stres,2,Lekki
Średni,8730,Stres,3,Średni
Średni,7079,Depresja,3,Średni
Średni,7048,Niepokój,3,Średni
Poważny,6113,Niepokój,4,Poważny


In [0]:
d5.select(f.col("Zaburzenie"), f.col("Poziom"), f.col("count"))\
    .join(d5.groupBy(f.col("Zaburzenie")).sum("count"), on = "Zaburzenie", how = "left")\
    .withColumn("Procent", 100*f.col("count")/f.col("sum(count)"))\
    .filter("Poziom like 'Bardzo poważny' OR Poziom like 'Poważny'")\
    .groupBy(f.col("Zaburzenie"))\
    .sum("Procent")\
    .withColumn("Procent", f.bround(f.col("sum(Procent)"), 0))\
    .select(f.col("Zaburzenie"), f.col("Procent"))\
    .display()

Zaburzenie,Procent
Stres,36.0
Niepokój,51.0
Depresja,50.0


Depresja i niepokój są częściej występującym zaburzeniem, niż stres - występują o około 15% częściej.

#### 3.1. Niepokój

In [0]:
d6 = d4.groupBy("Niepokój", "Orientacja").count()\
        .join(d4.groupBy("Orientacja").count().withColumnRenamed("count", "calosc"), on = "Orientacja", how = "left")\
    .filter(d4.Orientacja != "null")\
    .withColumn("Procent", f.bround(100*f.col("count")/f.col("calosc"),2))\
    .join(order, on = (f.col("Poziom")==f.col("Niepokój")), how = "left")\
    .orderBy(f.col("kolejność"))

In [0]:
d6.display()

Orientacja,Niepokój,count,calosc,Procent,kolejność,Poziom
Aseksualizm,Brak,410,1976,20.75,1,Brak
Heteroseksualizm,Brak,6853,24269,28.24,1,Brak
Biseksualizm,Brak,832,5006,16.62,1,Brak
Inna,Brak,632,3460,18.27,1,Brak
Homoseksualizm,Brak,419,1955,21.43,1,Brak
Homoseksualizm,Lekki,117,1955,5.98,2,Lekki
Aseksualizm,Lekki,134,1976,6.78,2,Lekki
Inna,Lekki,200,3460,5.78,2,Lekki
Biseksualizm,Lekki,273,5006,5.45,2,Lekki
Heteroseksualizm,Lekki,1819,24269,7.5,2,Lekki


In [0]:
d6.filter("Poziom like 'Poważny' OR Poziom like 'Bardzo poważny'")\
.select(f.col("Orientacja"), f.col("Poziom"), f.col("Procent"))\
.groupBy(f.col("Orientacja"))\
.sum("Procent")\
.withColumn("Procent", f.bround("sum(Procent)", 0))\
.select(f.col("Orientacja"), f.col("Procent"))\
.orderBy("Procent")\
.display()

Orientacja,Procent
Heteroseksualizm,46.0
Aseksualizm,56.0
Homoseksualizm,56.0
Inna,59.0
Biseksualizm,61.0


Najmniej narażoną na odczuwanie niepokoju grupą osób są osoby heteroseksualne - niepokój odczuwa o ponad 10% mniej heteroseksualnych osób, niż osób innej orientacji.

In [0]:
d4.groupBy("Niepokój", "Płeć").count()\
.filter(d4.Płeć != "null")\
.join(order, on = (f.col("Poziom")==f.col("Niepokój")), how = "left")\
.orderBy(f.col("kolejność"))\
.display()

Niepokój,Płeć,count,kolejność,Poziom
Brak,Inna,68,1,Brak
Brak,Mężczyzna,2973,1,Brak
Brak,Kobieta,6670,1,Brak
Lekki,Inna,26,2,Lekki
Lekki,Mężczyzna,679,2,Lekki
Lekki,Kobieta,2055,2,Lekki
Średni,Kobieta,5338,3,Średni
Średni,Inna,87,3,Średni
Średni,Mężczyzna,1607,3,Średni
Poważny,Kobieta,4764,4,Poważny


Grupą najbardziej narażoną na odczuwanie niepokoju są osoby, które identyfikują swoją płeć jako "Inna", grupą najmniej narażoną są mężczyźni:
* około 25% mniej mężczyzn odczuwa niepokój niż osoby, które nie identyfikują się jako kobieta/mężczyzna
* około 15 % więcej kobiet odczuwa niepokój niż mężczyzn

In [0]:
d4.groupBy("Niepokój", "Wykształcenie").count()\
.filter(d4.Wykształcenie != "null")\
.join(order, on = (f.col("Poziom")==f.col("Niepokój")), how = "left")\
.join(order_edu, on = (f.col("Wykształcenie")==f.col("wykszt")), how = "left")\
.orderBy(f.col("kolejność"), f.col("kolejność_wykształcenie"))\
.display()

Niepokój,Wykształcenie,count,kolejność,Poziom,kolejność_wykształcenie,wykszt
Brak,Podstawowe,604,1,Brak,1,Podstawowe
Brak,Średnie,3100,1,Brak,2,Średnie
Brak,Wyższe 1 stopnia,4095,1,Brak,3,Wyższe 1 stopnia
Brak,Wyższe 2 stopnia,1795,1,Brak,4,Wyższe 2 stopnia
Lekki,Podstawowe,201,2,Lekki,1,Podstawowe
Lekki,Średnie,983,2,Lekki,2,Średnie
Lekki,Wyższe 1 stopnia,1122,2,Lekki,3,Wyższe 1 stopnia
Lekki,Wyższe 2 stopnia,408,2,Lekki,4,Wyższe 2 stopnia
Średni,Podstawowe,628,3,Średni,1,Podstawowe
Średni,Średnie,2597,3,Średni,2,Średnie


Im wyższe wykształcenie, tym mniej osób odczuwa niepokój - między osobami posiadającymi wykształcenie podstawowe, a wykształcenie wyższe 2 stopnia różnica wynosi aż 27% (biorąc pod uwagę poziom niepokoju poważny oraz bardzo poważny).

In [0]:
d4.groupBy("Niepokój", "Stan_cywilny").count()\
.filter(d4.Stan_cywilny != "null")\
.join(order, on = (f.col("Poziom")==f.col("Niepokój")), how = "left")\
.orderBy(f.col("kolejność"))\
.display()

Niepokój,Stan_cywilny,count,kolejność,Poziom
Brak,Panna/Kawaler,7378,1,Brak
Brak,Zamężna/Żonaty,1891,1,Brak
Brak,Rozwiedziona/Rozwiedziony,416,1,Brak
Lekki,Rozwiedziona/Rozwiedziony,80,2,Lekki
Lekki,Zamężna/Żonaty,353,2,Lekki
Lekki,Panna/Kawaler,2315,2,Lekki
Średni,Panna/Kawaler,6070,3,Średni
Średni,Rozwiedziona/Rozwiedziony,186,3,Średni
Średni,Zamężna/Żonaty,768,3,Średni
Poważny,Zamężna/Żonaty,496,4,Poważny


Grupą najbardziej narażoną na niepokój są panny/kawalerowie, a najmniej narażone są osoby będące w związku małżeńskim. Biorąc pod uwagę poziom niepokoju poważny i bardzo poważny:
* około 15% mniej rozwodników odczuwa niepokój, niż osoby które nigdy nie były w związku małżeńskim
* około 25% mniej osób będących w związku małżeńskim odczuwa niepokój, w porównaiu do osoby które nigdy w związku małżeńskim nie były

In [0]:
d4.createOrReplaceTempView("tmp_d4")

In [0]:
%sql

SELECT `Niepokój`, tab1.Wiek, round(100*count/calosc,2) AS Procent 
FROM ((SELECT `Niepokój`, Wiek, count(*) AS count FROM tmp_d4 GROUP BY `Niepokój`, Wiek) tab1 
        JOIN (SELECT Wiek, count(*) AS calosc FROM tmp_d4 GROUP BY Wiek) tab2 ON tab1.Wiek = tab2.Wiek)
WHERE calosc > 1 and (`Niepokój` like 'Poważny' OR `Niepokój` like 'Bardzo poważny')
SORT BY Wiek

Niepokój,Wiek,Procent
Poważny,13,17.59
Bardzo poważny,13,57.67
Poważny,14,13.83
Bardzo poważny,14,58.95
Poważny,15,16.18
Bardzo poważny,15,51.87
Poważny,16,17.48
Bardzo poważny,16,47.95
Bardzo poważny,17,45.26
Poważny,17,15.98


Na niepokój zdecydowanie bardziej są narażone osoby młode. U osób w wieku 13 do 35 lat następuje stopniowy spadek odczuwania niepokoju. Pomiędzy osobami w wieku 13, a 35 lat różnica wynosi 40%. Później poziom niepokoju utrzymuje się na zbliżonym poziomie.

#### 3.2. Depresja

In [0]:
d4.groupBy("Depresja", "Płeć").count()\
.filter(d4.Płeć != "null")\
.join(order, on = (f.col("Poziom")==f.col("Depresja")), how = "left")\
.orderBy(f.col("kolejność"))\
.display()

Depresja,Płeć,count,kolejność,Poziom
Brak,Inna,57,1,Brak
Brak,Mężczyzna,2408,1,Brak
Brak,Kobieta,7295,1,Brak
Lekki,Inna,23,2,Lekki
Lekki,Mężczyzna,631,2,Lekki
Lekki,Kobieta,2213,2,Lekki
Średni,Kobieta,5476,3,Średni
Średni,Inna,84,3,Średni
Średni,Mężczyzna,1505,3,Średni
Poważny,Kobieta,4968,4,Poważny


Grupą osób które są najbardziej narażone na odczuwanie depresji są osoby, które nie identyfikują się ani jako kobieta ani jako mężczyzna - jest to około 20% więcej osób w porównaniu tych, które określają swoją płeć jako męska lub żeńska.

In [0]:
d7 = d4.groupBy("Depresja", "Wyznanie").count()\
        .join(d4.groupBy("Wyznanie").count().withColumnRenamed("count", "calosc"), on = "Wyznanie", how = "left")\
    .filter(d4.Wyznanie != "null")\
    .withColumn("Procent", f.bround(100*f.col("count")/f.col("calosc"),2))\
    .join(order, on = (f.col("Poziom")==f.col("Depresja")), how = "left")\
    .orderBy(f.col("kolejność"))

In [0]:
d7.display()

Wyznanie,Depresja,count,calosc,Procent,kolejność,Poziom
Sikh,Brak,13,64,20.31,1,Brak
Ateista,Brak,733,3845,19.06,1,Brak
Inne,Brak,376,1804,20.84,1,Brak
Chrześcijanin (Protestant),Brak,451,1544,29.21,1,Brak
Chrześcijanin (Inny odłam),Brak,571,2254,25.33,1,Brak
Agnostyk,Brak,654,3245,20.15,1,Brak
Buddysta,Brak,171,527,32.45,1,Brak
Hindus,Brak,215,700,30.71,1,Brak
Chrześcijanin (Mormon),Brak,27,122,22.13,1,Brak
Chrześcijanin (Katolik),Brak,757,3097,24.44,1,Brak


In [0]:
d7.filter("Poziom like 'Poważny' OR Poziom like 'Bardzo poważny'")\
.select(f.col("Wyznanie"), f.col("Poziom"), f.col("Procent"))\
.groupBy(f.col("Wyznanie"))\
.sum("Procent")\
.withColumn("Procent", f.bround("sum(Procent)", 0))\
.select(f.col("Wyznanie"), f.col("Procent"))\
.orderBy("Procent")\
.display()

Wyznanie,Procent
Buddysta,42.0
Chrześcijanin (Protestant),45.0
Muzułmanin,47.0
Hindus,48.0
Żyd,50.0
Chrześcijanin (Inny odłam),51.0
Chrześcijanin (Katolik),51.0
Chrześcijanin (Mormon),54.0
Sikh,55.0
Inne,56.0


Najbardziej narażeni na depresję są ateiści oraz agnostycy, a najmniej buddyści - różnica między tymi skrajnymi wartościami wynosi 18%.

In [0]:
d4.groupBy("Depresja", "Stan_cywilny").count()\
.filter(d4.Stan_cywilny != "null")\
.join(order, on = (f.col("Poziom")==f.col("Depresja")), how = "left")\
.orderBy(f.col("kolejność"))\
.display()

Depresja,Stan_cywilny,count,kolejność,Poziom
Brak,Panna/Kawaler,7699,1,Brak
Brak,Zamężna/Żonaty,1709,1,Brak
Brak,Rozwiedziona/Rozwiedziony,325,1,Brak
Lekki,Rozwiedziona/Rozwiedziony,70,2,Lekki
Lekki,Zamężna/Żonaty,377,2,Lekki
Lekki,Panna/Kawaler,2414,2,Lekki
Średni,Panna/Kawaler,6068,3,Średni
Średni,Rozwiedziona/Rozwiedziony,178,3,Średni
Średni,Zamężna/Żonaty,796,3,Średni
Poważny,Zamężna/Żonaty,584,4,Poważny


Najmniej narażone na depresje są osoby, które są w związku małżeńskim:
* około 20% mniej osób w związku małżeńskim jest narażonych na depresję, niż osób które nigdy w takim związku nie były
* około 15% więcej osób po rozwodzie jest narażonych na depresję, niż osób w związku małżeńskim

#### 3.3. Stres

In [0]:
d4.groupBy("Stres", "Wykształcenie").count()\
.filter(d4.Wykształcenie != "null")\
.join(order, on = (f.col("Poziom")==f.col("Stres")), how = "left")\
.join(order_edu, on = (f.col("Wykształcenie")==f.col("wykszt")), how = "left")\
.orderBy(f.col("kolejność"), f.col("kolejność_wykształcenie"))\
.display()

Stres,Wykształcenie,count,kolejność,Poziom,kolejność_wykształcenie,wykszt
Brak,Podstawowe,764,1,Brak,1,Podstawowe
Brak,Średnie,3943,1,Brak,2,Średnie
Brak,Wyższe 1 stopnia,4979,1,Brak,3,Wyższe 1 stopnia
Brak,Wyższe 2 stopnia,1957,1,Brak,4,Wyższe 2 stopnia
Lekki,Podstawowe,434,2,Lekki,1,Podstawowe
Lekki,Średnie,1784,2,Lekki,2,Średnie
Lekki,Wyższe 1 stopnia,1985,2,Lekki,3,Wyższe 1 stopnia
Lekki,Wyższe 2 stopnia,649,2,Lekki,4,Wyższe 2 stopnia
Średni,Podstawowe,923,3,Średni,1,Podstawowe
Średni,Średnie,3379,3,Średni,2,Średnie


Im wyższe wykształcenie, tym mniej osób odczuwa stres - różnica między osobami posiadającymi wykształcenie podstawowe, a wykształcenie 2 stopnia wynosi 20%.

In [0]:
d8 = d4.groupBy("Stres", "Wyznanie").count()\
        .join(d4.groupBy("Wyznanie").count().withColumnRenamed("count", "calosc"), on = "Wyznanie", how = "left")\
    .filter(d4.Wyznanie != "null")\
    .withColumn("Procent", f.bround(100*f.col("count")/f.col("calosc"),2))\
    .join(order, on = (f.col("Poziom")==f.col("Stres")), how = "left")\
    .orderBy(f.col("kolejność"))

In [0]:
d8.display()

Wyznanie,Stres,count,calosc,Procent,kolejność,Poziom
Sikh,Brak,17,64,26.56,1,Brak
Ateista,Brak,1068,3845,27.78,1,Brak
Inne,Brak,450,1804,24.94,1,Brak
Chrześcijanin (Protestant),Brak,567,1544,36.72,1,Brak
Chrześcijanin (Inny odłam),Brak,634,2254,28.13,1,Brak
Agnostyk,Brak,879,3245,27.09,1,Brak
Hindus,Brak,248,700,35.43,1,Brak
Buddysta,Brak,190,527,36.05,1,Brak
Chrześcijanin (Mormon),Brak,32,122,26.23,1,Brak
Chrześcijanin (Katolik),Brak,840,3097,27.12,1,Brak


In [0]:
d8.filter("Poziom like 'Poważny' OR Poziom like 'Bardzo poważny'")\
.select(f.col("Wyznanie"), f.col("Poziom"), f.col("Procent"))\
.groupBy(f.col("Wyznanie"))\
.sum("Procent")\
.withColumn("Procent", f.bround("sum(Procent)", 0))\
.select(f.col("Wyznanie"), f.col("Procent"))\
.orderBy("Procent")\
.display()

Wyznanie,Procent
Buddysta,29.0
Chrześcijanin (Protestant),31.0
Hindus,34.0
Muzułmanin,35.0
Żyd,35.0
Sikh,36.0
Agnostyk,37.0
Chrześcijanin (Katolik),38.0
Ateista,38.0
Chrześcijanin (Inny odłam),39.0


Najbardziej narażeni na stres są mormoni - stres odczuwa ponad 10% więcej osób, niż osób z pozostałych wyznań.
Najmniej narażeni na stres są buddyści - różnica między buddystami i mormonami wynosi aż 22%.

### 4. Podsumowanie

Dokonując analizy wyników testu, można wyodrębnić kilka grup najbardziej/najmniej podatnych na takie zaburzenia jak depresja, stres i niepokój. W podziale na kategorie:
* **Płeć**: Na niepokój oraz na depresję najbardziej są narażone osoby, które nie identyfikują swojej płci jako kobieta ani jako mężczyzna. Mężczyźni są mniej podatni na niepokój niż kobiety.
* **Wiek**: Osoby nastoletnie zdecydowanie bardziej odczuwają niepokój, niż osoby będą powyżej 30 roku życia.
* **Wyznanie**: Buddiści są grupą, która w najmniejszym stopniu odczuwa depresję oraz stres. Najczęściej depresję odczuwają ateiści oraz agnostycy. Natomiast stres najczęściej odczuwają mormoni.
* **Orientacja**: Najbardziej podatni na niepokój są biseksualiści. Najmniej natomiast heteroseksualiści.
* **Stan cywilny**: Najmniejszą grupą osób odczuwająca niepokój oraz depresję są ludzie, którzy są w związku małżeńskim.
* **Wykształcenie**: Im wyższe wykształcenie, tym mniej ludzi odczuwa niepokój oraz stres.